In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

import numpy as np

#학습셋http://localhost:8888/tree의 변형을 설정하는 부분
train_datagen = ImageDataGenerator(rescale=1./255, #주어진 이미지의 크기를 설정
                                   horizontal_flip = True, #수평대칭 이미지를 50%확률로 만듬
                                   width_shift_range = 0.1, #전체크기의 15%범위에서 좌우로 이동
                                   height_shift_range=0.1, #전체크기의 15%범위에서 상하로 이동
                                   rotation_range=5, #정해진 각도만큼 회전
                                   shear_range = 0.7, #좌표 하나를 고정시키고 나머지를 이동시킴 
                                   zoom_range = 1.2, #확대 / 축소
                                   vertical_flip = True, #수직 대칭 이미지를 만듬
                                   fill_mode = 'nearest' #빈 공간을 채우는 방법
                                   # nearest : 비슷한 색으로
                                  )
train_generator = train_datagen.flow_from_directory(
    '../data/train', #학습셋이 있는 폴더 위치
    target_size = (150,150),
    batch_size = 5,
    class_mode = 'binary'
)

#테스트셋
#이미지 부풀리기 과정을 진행안함
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '../data/train', #학습셋이 있는 폴더 위치
    target_size = (150,150),
    batch_size = 5,
    class_mode = 'binary'
)

#CNN 모델을 만들어서 적용
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))                                                                                                       

Found 160 images belonging to 2 classes.
Found 160 images belonging to 2 classes.




In [18]:
#모델실행
model.compile(loss = 'binary_crossentropy', \
              optimizer=optimizers.Adam(learning_rate = 0.0002), metrics='accuracy')

#학습의 조기중단
early_stopping_callback = EarlyStopping(monitor='var_loss', patience=5)

#모델학습
history = model.fit(train_generator, epochs=100, validation_data = test_generator,
                   validation_steps = 10, callbacks=[early_stopping_callback])

Epoch 1/100


32/32 [==============================] - 4s 100ms/step - loss: 1.0924 - accuracy: 0.4437 - val_loss: 0.7728 - val_accuracy: 0.3400
Epoch 2/100
32/32 [==============================] - 3s 82ms/step - loss: 0.7242 - accuracy: 0.5437 - val_loss: 0.6879 - val_accuracy: 0.5800
Epoch 3/100
32/32 [==============================] - 3s 83ms/step - loss: 0.6895 - accuracy: 0.5437 - val_loss: 0.6928 - val_accuracy: 0.4800
Epoch 4/100
32/32 [==============================] - 3s 83ms/step - loss: 0.7035 - accuracy: 0.5250 - val_loss: 0.6953 - val_accuracy: 0.4200
Epoch 5/100
32/32 [==============================] - 3s 82ms/step - loss: 0.7031 - accuracy: 0.4688 - val_loss: 0.6911 - val_accuracy: 0.5400
Epoch 6/100
32/32 [==============================] - 3s 86ms/step - loss: 0.6930 - accuracy: 0.4750 - val_loss: 0.6915 - val_accuracy: 0.5200
Epoch 7/100
32/32 [==============================] - 3s 84ms/step - loss: 0.6922 - accuracy: 0.5250 - val_loss: 0.6903 - val_accuracy: 0.5400
Epo

In [25]:
#전이학습
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers, Input, models, layers, metrics
from tensorflow.keras.applications import VGG16

#내가가진 이미지를 학습시키기
#학습셋의 변형을 설정
train_datagen = ImageDataGenerator(rescale=1./255, #주어진 이미지의 크기를 설정
                                   horizontal_flip = True, #수평대칭 이미지를 50%확률로 만듬
                                   width_shift_range = 0.1, #전체크기의 15%범위에서 좌우로 이동
                                   height_shift_range=0.1, #전체크기의 15%범위에서 상하로 이동
                                   rotation_range=5, #정해진 각도만큼 회전
                                   shear_range = 0.7, #좌표 하나를 고정시키고 나머지를 이동시킴 
                                   zoom_range = 1.2, #확대 / 축소
                                   vertical_flip = True, #수직 대칭 이미지를 만듬
                                   fill_mode = 'nearest' #빈 공간을 채우는 방법
                                   # nearest : 비슷한 색으로
                                  )
train_generator = train_datagen.flow_from_directory(
    '../data/train', #학습셋이 있는 폴더 위치
    target_size = (150,150),
    batch_size = 5,
    class_mode = 'binary'
)

#테스트셋
#이미지 부풀리기 과정을 진행안함
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '../data/train', #학습셋이 있는 폴더 위치
    target_size = (150,150),
    batch_size = 5,
    class_mode = 'binary'
)

#VGG16 모델
transfer_model = VGG16(weights='imagenet', include_top=False)
transfer_model.trainable = False
#transfer_model.summary()

#우리의 모델을 설정
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Flatten())

finetune_model.add(Dense(64))
finetune_model.add(Activation('relu'))
finetune_model.add(Dropout(0.5))


finetune_model.add(Dense(1))
finetune_model.add(Activation('sigmoid'))
finetune_model.summary()

Found 160 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)

In [ ]:
#모델실행
finetune_model.compile(loss = 'binary_crossentropy', \
              optimizer=optimizers.Adam(learning_rate = 0.0002), metrics=['accuracy'])

#학습의 조기중단
early_stopping_callback = EarlyStopping(monitor='var_loss', patience=5)

#모델학습
history = finetune_model.fit(train_generator, epochs=100, validation_data = test_generator,
                   validation_steps = 10, callbacks=[early_stopping_callback])

Epoch 1/100
32/32 [==============================] - 7s 217ms/step - loss: 6.2777 - accuracy: 0.0000e+00 - val_loss: 5.0310 - val_accuracy: 0.0000e+00
Epoch 2/100
32/32 [==============================] - 6s 198ms/step - loss: 6.2778 - accuracy: 0.0000e+00 - val_loss: 6.3499 - val_accuracy: 0.0000e+00
Epoch 3/100
32/32 [==============================] - 7s 212ms/step - loss: 6.3050 - accuracy: 0.0000e+00 - val_loss: 5.9154 - val_accuracy: 0.0000e+00
Epoch 4/100
32/32 [==============================] - 7s 227ms/step - loss: 6.2991 - accuracy: 0.0000e+00 - val_loss: 6.5845 - val_accuracy: 0.0000e+00
Epoch 5/100
32/32 [==============================] - 8s 255ms/step - loss: 6.2813 - accuracy: 0.0000e+00 - val_loss: 6.1430 - val_accuracy: 0.0000e+00
Epoch 6/100
32/32 [==============================] - 7s 232ms/step - loss: 6.2345 - accuracy: 0.0000e+00 - val_loss: 7.0113 - val_accuracy: 0.0000e+00
Epoch 7/100
32/32 [==============================] - 6s 203ms/step - loss: 6.2856 - accuracy: 